In [0]:
# ✅ Step 1: Define your Azure Storage info
storage_account_name = "dmodemoaccount123"
storage_account_key = ""  # Replace with actual key from Azure Portal

# ✅ Step 2: Set Spark config for accessing ADLS Gen2
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

In [0]:
processName = dbutils.widgets.get("prm_processName")

nextSourceFileDateSql = f"""
SELECT NVL(DATE_ADD(MAX(CAST(PROCESSED_FILE_TABLE_DATE AS DATE)), 1),'2023-01-01') AS NEXT_SOURCE_FILE_DATE
FROM default.deltalakehouse_process_runs
WHERE process_name = '{processName}'
  AND PROCESS_STATUS = 'Completed';
"""

print(nextSourceFileDateSql)

spark.sql(nextSourceFileDateSql)

nextSourceFileDateDF=spark.sql(nextSourceFileDateSql)
nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE'] # converting to python string
print(nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE']) 



SELECT NVL(DATE_ADD(MAX(CAST(PROCESSED_FILE_TABLE_DATE AS DATE)), 1),'2023-01-01') AS NEXT_SOURCE_FILE_DATE
FROM default.deltalakehouse_process_runs
WHERE process_name = 'dailyPricingSourceIngest'
  AND PROCESS_STATUS = 'Completed';



{"ts": "2025-08-04 08:53:24.230", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[TABLE_OR_VIEW_NOT_FOUND] The table or view `default`.`deltalakehouse_process_runs` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.\nTo tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS. SQLSTATE: 42P01", "context": {"errorClass": "TABLE_OR_VIEW_NOT_FOUND"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o404.sql.\n: org.apache.spark.sql.catalyst.ExtendedAnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `default`.`deltalakehouse_process_runs` cannot be found. Verify the spelling and correctness of the schema and catalog.\nIf you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and ca

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8943734347796717>, line 12
      3 nextSourceFileDateSql = f"""
      4 SELECT NVL(DATE_ADD(MAX(CAST(PROCESSED_FILE_TABLE_DATE AS DATE)), 1),'2023-01-01') AS NEXT_SOURCE_FILE_DATE
      5 FROM default.deltalakehouse_process_runs
      6 WHERE process_name = '{processName}'
      7   AND PROCESS_STATUS = 'Completed';
      8 """
     10 print(nextSourceFileDateSql)
---> 12 spark.sql(nextSourceFileDateSql)
     14 nextSourceFileDateDF=spark.sql(nextSourceFileDateSql)
     15 nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE'] # converting to python string

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_suc

In [0]:
from datetime import datetime
dbutils.widgets.text('prm_dailyPricingSourceFileDate', '')
print(datetime.strptime(dbutils.widgets.get('prm_dailyPricingSourceFileDate'),'%Y-%m-%d').strftime('%d%m%Y'))

In [0]:
sourceFileURL='https://retailpricing.blob.core.windows.net/labs/lab1/PW_MW_DR_01012023.csv'
bronzelayerCSVFilePath='abfss://working-labs@dmodemoaccount123.dfs.core.windows.net/bronze/daily-pricing/csv'

In [0]:
dailyPricingSourceBaseURL='https://retailpricing.blob.core.windows.net/'
dailyPricingSourceFolder='daily-pricing/'
#dailyPricingSourceFileDate=dbutils.widgets.get('prm_dailyPricingSourceFileDate')
dailyPricingSourceFileDate=nextSourceFileDateDF.select("NEXT_SOURCE_FILE_DATE").collect()[0]['NEXT_SOURCE_FILE_DATE']; dailyPricingSourceFileDate=str(dailyPricingSourceFileDate); dailyPricingSourceFileDate=datetime.strptime(dailyPricingSourceFileDate, '%Y-%m-%d').strftime('%d%m%Y')
dailyPricingSourceFileName=f"PW_MW_DR_{dailyPricingSourceFileDate}.csv"
print(dailyPricingSourceFileName)

dailyPricingSinkLayerName='bronze'
dailyPricingSinkStorageAccountName='dmodemoaccount123'
dailyPricingSinkFolderName='daily-pricing'

In [0]:
import pandas as pds

In [0]:
dailyPricingSourceURL=dailyPricingSourceBaseURL+dailyPricingSourceFolder+dailyPricingSourceFileName
print(dailyPricingSourceURL)
dailyPricingPandasDF=pds.read_csv(dailyPricingSourceURL)
display(dailyPricingPandasDF)

In [0]:
dailyPricingSparkDF=spark.createDataFrame(dailyPricingPandasDF)
#display(dailyPricingSparkDF)

In [0]:
from pyspark.sql.functions import current_timestamp
dailyPricingSynkFolderPath=f"abfss://{dailyPricingSinkLayerName}@{dailyPricingSinkStorageAccountName}.dfs.core.windows.net/{dailyPricingSinkFolderName}"
print(dailyPricingSynkFolderPath)
(dailyPricingSparkDF
 .withColumn("source_file_load_date",current_timestamp())
 .write
 .mode("append")
 .option("header","true")
 .csv(dailyPricingSynkFolderPath)
)


In [0]:
#dbutils.fs.ls(dailyPricingSynkFolderPath)

# dbutils.fs.rm(dailyPricingSynkFolderPath, recurse=True)

In [0]:
%sql
-- CREATE SCHEMA IF NOT EXISTS processrunlogs;
-- CREATE TABLE IF NOT EXISTS processrunlogs.DELTALAKEHOUE_PROCESS_RUNS(
--     PROCESS_NAME STRING,
--    PROCESSED_FILE_TABLE_DATE DATE,
--    PROCESS_STATUS STRING
   
-- )
-- delete while refactor it

In [0]:
%sql

SELECT DATE_ADD(MAX(CAST(PROCESSED_FILE_TABLE_DATE AS DATE)), 1) AS NEXT_SOURCE_FILE_DATE
FROM processrunlogs.deltalakehouse_process_runs
WHERE process_name = 'dailyPricingSourceIngest'
  AND PROCESS_STATUS = 'Completed';

  -- delete while refactor it

In [0]:
processName = 'dailyPricingSourceIngest'
processFileDate = dbutils.widgets.get('prm_dailyPricingSourceFileDate')
processStatus = 'Completed'

# Compose SQL insert without catalog prefix
processInsertSql = f"""
INSERT INTO processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS 
(PROCESS_NAME, PROCESSED_FILE_TABLE_DATE, PROCESS_STATUS)
VALUES ('{processName}', '{processFileDate}', '{processStatus}')
"""

spark.sql(processInsertSql)

In [0]:
%sql
SELECT  * FROM processrunlogs.deltalakehouse_process_runs

In [0]:

%sql
SELECT DATE_ADD(MAX(CAST(PROCESSED_FILE_TABLE_DATE AS DATE)), 1) AS NEXT_SOURCE_FILE_DATE
FROM processrunlogs.deltalakehouse_process_runs
WHERE process_name = 'dailyPricingSourceIngest'
  AND PROCESS_STATUS = 'Completed';


In [0]:
dbutils.fs.rm(dailyPricingSynkFolderPath, recurse=True) # delete files in the folder

In [0]:
%sql
delete from processrunlogs.DELTALAKEHOUSE_PROCESS_RUNS